이 실습은 Runpod에서 A100 SXM GPU 1개로 진행되었습니다.

## 1. 필요한 패키지 설치

In [2]:
%pip install "torch==2.4.0"
%pip install "transformers==4.45.1" "datasets==3.0.1" "accelerate==0.34.2" "trl==0.11.1" "peft==0.13.0"


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 2. 데이터 전처리

In [3]:
from datasets import load_dataset, Dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer

In [4]:
!wget https://github.com/llm-fine-tuning/LLaMA-Factory/raw/refs/heads/main/data/esg_binary_classification_train.json
!wget https://github.com/llm-fine-tuning/LLaMA-Factory/raw/refs/heads/main/data/esg_binary_classification_test.json

--2025-06-04 07:47:40--  https://github.com/llm-fine-tuning/LLaMA-Factory/raw/refs/heads/main/data/esg_binary_classification_train.json
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/llm-fine-tuning/LLaMA-Factory/refs/heads/main/data/esg_binary_classification_train.json [following]
--2025-06-04 07:47:41--  https://raw.githubusercontent.com/llm-fine-tuning/LLaMA-Factory/refs/heads/main/data/esg_binary_classification_train.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10738972 (10M) [application/octet-stream]
Saving to: ‘esg_binary_classification_train.json.2’

esg_binary_classifi 100%[===

In [5]:
import json
import random

# 1. JSON 파일 로드
def load_dataset(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        dataset = json.load(f)
    return dataset

# 2. 데이터 로드
train_dataset = load_dataset('esg_binary_classification_train.json')
test_dataset = load_dataset('esg_binary_classification_test.json')

print(f"학습 데이터 개수: {len(train_dataset)}")
print(f"테스트 데이터 개수: {len(test_dataset)}")

학습 데이터 개수: 3728
테스트 데이터 개수: 932


In [6]:
# 3. 데이터 구조 확인
print("\n=== 데이터 구조 확인 ===")
print("첫 번째 학습 데이터 샘플:")
print(json.dumps(train_dataset[0], ensure_ascii=False, indent=2))


=== 데이터 구조 확인 ===
첫 번째 학습 데이터 샘플:
{
  "instruction": "LG에너지솔루션, 배터리 관리 분야 새 얼굴 '비 어라운드' 만들었다 LG에너지솔루션의 배터리 관리 토털 설루션(BMTS) 브랜드 'B.around'(비.어라운드) 로고. LG에너지솔루션 제공  잇단 화재로 전기차 배터리에 불안과 공포(전기차 포비아)가 확산한 가운데 LG에너지솔루션(LG엔솔)이 배터리 관리 토털 설루션(BMTS) 브랜드를 만들고 사업 확장에 나선다. 전기차 배터리 생산을 넘어 소프트웨어 부문 강화로 수익원을 넓히려는 움직임으로 풀이된다.  LG엔솔은 25일 '항상 고객의 곁에'(Be around your side)란 뜻의 BMTS 브랜드 'B.around'(비.어라운드)를 공개했다. 어떤 상황에서든 배터리 상태를 실시간 모니터링하는 LG엔솔 BMTS의 기술 경쟁력을 상징한다는 설명이다.  비.어라운드 제품에는 기존 배터리 관리 시스템(BMS)에 클라우드와 인공지능(AI) 기술을 결합해 화재방지 등을 위한 안전 진단, 퇴화·수명 예측 등을 더 쉽게 할 계획이다. 배터리 불량 유형을 사전에 진단하고 퇴화 상태를 점검하는 소프트웨어를 통해 오랫동안 쓸 수 있게 할 방침이다.  이달훈 LG에너지솔루션 BMS개발센터장 상무는 \"오랜 기간 안전 진단 시스템을 위해 노력했고 이제는 배터리 관리 소프트웨어 사업으로 확장해 배터리의 건강한 사용을 위해 설루션을 제공하는 기업이 되려고 한다\"고 말했다.  김청환 기자 chk@hankookilbo.com",
  "input": "",
  "output": "근거: LG에너지솔루션이 배터리 관리 토털 솔루션(BMTS) 브랜드인 'B.around'를 출시한 것은 ESG와 관련이 있습니다. 사회 측면에서는 화재 방지와 같은 안전 진단을 포함한 배터리 관리 강화는 제품 책임 및 안전을 향상시키기 위한 노력으로 볼 수 있습니다. 이는 고객의 신뢰를 높이고 제품의 안전성을 확보하는 사회적 책임 경영과 관련이 깊습니다. 또한

In [7]:
# 4. OpenAI format으로 데이터 변환을 위한 함수
system_prompt = '''1. 상장사의 뉴스를 보고 ESG 관련 기사인지 판단하시오.
2. ESG 각각에 대한 핵심 키워드는 아래의 10개 키워드를 참고해서 판단하시오.
- 환경 (Environmental) : 탄소 배출 감소 (Carbon Emission Reduction), 재생 에너지 사용 (Renewable Energy Usage), 자원 효율성 (Resource Efficiency), 폐기물 관리 (Waste Management), 친환경 제품 개발 (Eco-Friendly Product Development), 기후 변화 대응 (Climate Change Mitigation), 오염 방지 (Pollution Prevention), 생물 다양성 보호 (Biodiversity Protection), 물 관리 (Water Management)
순환 경제 (Circular Economy)
- 사회 (Social) : 노동 인권 보호 (Labor Rights Protection), 다양성 및 포용성 (Diversity and Inclusion), 직원 안전 및 건강 (Employee Health & Safety), 공정한 노동 관행 (Fair Labor Practices), 커뮤니티 참여 및 개발 (Community Engagement & Development), 제품 책임 및 안전 (Product Responsibility & Safety), 공급망 책임 (Supply Chain Responsibility), 데이터 보호 및 프라이버시 (Data Protection & Privacy), 인재 개발 및 교육 (Talent Development & Education), 고객 만족도 (Customer Satisfaction)
- 지배구조 (Governance) : 이사회 독립성 (Board Independence), 윤리적 경영 (Ethical Management), 주주 권리 보호 (Shareholder Rights Protection), 내부 통제 및 감사 (Internal Controls & Audits), 리스크 관리 (Risk Management), 투명한 정보 공개 (Transparent Disclosure), 기업 윤리 및 부패 방지 (Corporate Ethics & Anti-Corruption), 경영진 보상 구조 (Executive Compensation Structure), 컴플라이언스 (Regulatory Compliance), 이해관계자 참여 (Stakeholder Engagement)
3. 입력이 주어지면 근거를 작성하고 ESG 관련 기사이면 => True 아니면 => False를 작성합니다.
4. 답변은 항상 '=> True' 또는 '=> False'로 끝나야 하며 그 뒤에 아무 것도 적지마세요.
5. 지역의 정책이나 장소에 대한 내용이거나, 개인의 사건과 같이 특정 기업에 대한 이야기가 아닌 경우에는 아니라고 판단하세요. 저는 상장 '기업'의 내용에 집중하고 있습니다.
6. 기업의 경영권 분쟁과 같은 경우에는 단순히 개인의 이야기라고 볼 수 없습니다. 이 경우에도 ESG라고 판단할 수 있습니다.
7. 개인적인 부고의 기사는 ESG와 관련이 없다고 판단하세요.'''

def format_data(sample):
    return {
        "messages": [
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user", 
                "content": sample["instruction"]  # instruction을 user prompt로 사용
            },
            {
                "role": "assistant",
                "content": sample["output"]  # output을 assistant response로 사용
            }
        ]
    }

In [8]:
# 5. 전체 데이터셋을 OpenAI 포맷으로 변환
def convert_to_openai_format(dataset):
    formatted_data = []
    for sample in dataset:
        try:
            formatted_sample = format_data(sample)
            formatted_data.append(formatted_sample)
        except KeyError as e:
            print(f"키 오류 발생: {e}")
            print(f"샘플 데이터: {sample}")
            break
    return formatted_data

# 6. 데이터 변환 실행
print("\n=== OpenAI 포맷으로 변환 중... ===")
train_dataset = convert_to_openai_format(train_dataset)
test_datatset = convert_to_openai_format(test_dataset)

print(f"변환된 학습 데이터 개수: {len(train_dataset)}")
print(f"변환된 테스트 데이터 개수: {len(test_dataset)}")


=== OpenAI 포맷으로 변환 중... ===
변환된 학습 데이터 개수: 3728
변환된 테스트 데이터 개수: 932


In [9]:
# 임의의 345번 데이터 출력
train_dataset[345]

{'messages': [{'role': 'system',
   'content': "1. 상장사의 뉴스를 보고 ESG 관련 기사인지 판단하시오.\n2. ESG 각각에 대한 핵심 키워드는 아래의 10개 키워드를 참고해서 판단하시오.\n- 환경 (Environmental) : 탄소 배출 감소 (Carbon Emission Reduction), 재생 에너지 사용 (Renewable Energy Usage), 자원 효율성 (Resource Efficiency), 폐기물 관리 (Waste Management), 친환경 제품 개발 (Eco-Friendly Product Development), 기후 변화 대응 (Climate Change Mitigation), 오염 방지 (Pollution Prevention), 생물 다양성 보호 (Biodiversity Protection), 물 관리 (Water Management)\n순환 경제 (Circular Economy)\n- 사회 (Social) : 노동 인권 보호 (Labor Rights Protection), 다양성 및 포용성 (Diversity and Inclusion), 직원 안전 및 건강 (Employee Health & Safety), 공정한 노동 관행 (Fair Labor Practices), 커뮤니티 참여 및 개발 (Community Engagement & Development), 제품 책임 및 안전 (Product Responsibility & Safety), 공급망 책임 (Supply Chain Responsibility), 데이터 보호 및 프라이버시 (Data Protection & Privacy), 인재 개발 및 교육 (Talent Development & Education), 고객 만족도 (Customer Satisfaction)\n- 지배구조 (Governance) : 이사회 독립성 (Board Independence), 윤리적 경영 (Ethical Management), 주주 권리 보호 (Shar

In [10]:
# 리스트 형태에서 다시 Dataset 객체로 변경
print(type(train_dataset))
print(type(test_dataset))
train_dataset = Dataset.from_list(train_dataset)
test_dataset = Dataset.from_list(test_dataset)
print(type(train_dataset))
print(type(test_dataset))

<class 'list'>
<class 'list'>
<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>


In [11]:
train_dataset[0]

{'messages': [{'content': "1. 상장사의 뉴스를 보고 ESG 관련 기사인지 판단하시오.\n2. ESG 각각에 대한 핵심 키워드는 아래의 10개 키워드를 참고해서 판단하시오.\n- 환경 (Environmental) : 탄소 배출 감소 (Carbon Emission Reduction), 재생 에너지 사용 (Renewable Energy Usage), 자원 효율성 (Resource Efficiency), 폐기물 관리 (Waste Management), 친환경 제품 개발 (Eco-Friendly Product Development), 기후 변화 대응 (Climate Change Mitigation), 오염 방지 (Pollution Prevention), 생물 다양성 보호 (Biodiversity Protection), 물 관리 (Water Management)\n순환 경제 (Circular Economy)\n- 사회 (Social) : 노동 인권 보호 (Labor Rights Protection), 다양성 및 포용성 (Diversity and Inclusion), 직원 안전 및 건강 (Employee Health & Safety), 공정한 노동 관행 (Fair Labor Practices), 커뮤니티 참여 및 개발 (Community Engagement & Development), 제품 책임 및 안전 (Product Responsibility & Safety), 공급망 책임 (Supply Chain Responsibility), 데이터 보호 및 프라이버시 (Data Protection & Privacy), 인재 개발 및 교육 (Talent Development & Education), 고객 만족도 (Customer Satisfaction)\n- 지배구조 (Governance) : 이사회 독립성 (Board Independence), 윤리적 경영 (Ethical Management), 주주 권리 보호 (Shareholder Rights Protec

## 2. 모델 로드 및 템플릿 적용

In [12]:
# 허깅페이스 모델 ID
model_id = "allganize/Llama-3-Alpha-Ko-8B-Instruct" 

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [13]:
# 템플릿 적용
text = tokenizer.apply_chat_template(
    train_dataset[0]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

1. 상장사의 뉴스를 보고 ESG 관련 기사인지 판단하시오.
2. ESG 각각에 대한 핵심 키워드는 아래의 10개 키워드를 참고해서 판단하시오.
- 환경 (Environmental) : 탄소 배출 감소 (Carbon Emission Reduction), 재생 에너지 사용 (Renewable Energy Usage), 자원 효율성 (Resource Efficiency), 폐기물 관리 (Waste Management), 친환경 제품 개발 (Eco-Friendly Product Development), 기후 변화 대응 (Climate Change Mitigation), 오염 방지 (Pollution Prevention), 생물 다양성 보호 (Biodiversity Protection), 물 관리 (Water Management)
순환 경제 (Circular Economy)
- 사회 (Social) : 노동 인권 보호 (Labor Rights Protection), 다양성 및 포용성 (Diversity and Inclusion), 직원 안전 및 건강 (Employee Health & Safety), 공정한 노동 관행 (Fair Labor Practices), 커뮤니티 참여 및 개발 (Community Engagement & Development), 제품 책임 및 안전 (Product Responsibility & Safety), 공급망 책임 (Supply Chain Responsibility), 데이터 보호 및 프라이버시 (Data Protection & Privacy), 인재 개발 및 교육 (Talent Development & Education), 고객 만족도 (Customer Satisfaction)
- 지배구조 (Governance) : 이사회 독립성 (Board Independence), 윤리적 경영 (Ethical Management), 주주 권리 

## 3. LoRA와 SFTConfig 설정

In [14]:
peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.1,
        r=8,
        bias="none",
        target_modules=["q_proj", "v_proj"],
        task_type="CAUSAL_LM",
)

`lora_alpha`: LoRA(Low-Rank Adaptation)에서 사용하는 스케일링 계수를 설정합니다. LoRA의 가중치 업데이트가 모델에 미치는 영향을 조정하는 역할을 하며, 일반적으로 학습 안정성과 관련이 있습니다.

`lora_dropout`: LoRA 적용 시 드롭아웃 확률을 설정합니다. 드롭아웃은 과적합(overfitting)을 방지하기 위해 일부 뉴런을 랜덤하게 비활성화하는 정규화 기법입니다. 0.1로 설정하면 학습 중 10%의 뉴런이 비활성화됩니다.

`r`: LoRA의 랭크(rank)를 설정합니다. 이는 LoRA가 학습할 저차원 공간의 크기를 결정합니다. 작은 값일수록 계산 및 메모리 효율이 높아지지만 모델의 학습 능력이 제한될 수 있습니다.

`bias`: LoRA 적용 시 편향(bias) 처리 방식을 지정합니다. "none"으로 설정하면 편향이 LoRA에 의해 조정되지 않습니다. "all" 또는 "lora_only"와 같은 값으로 변경하여 편향을 조정할 수도 있습니다.

`target_modules`: LoRA를 적용할 특정 모듈(레이어)의 이름을 리스트로 지정합니다. 예제에서는 "q_proj"와 "v_proj"를 지정하여, 주로 Self-Attention 메커니즘의 쿼리와 값 프로젝션 부분에 LoRA를 적용합니다.

`task_type:` LoRA가 적용되는 작업 유형을 지정합니다. "CAUSAL_LM"은 Causal Language Modeling, 즉 시퀀스 생성 작업에 해당합니다. 다른 예로는 "SEQ2SEQ_LM"(시퀀스-투-시퀀스 언어 모델링) 등이 있습니다.

In [15]:
args = SFTConfig(
    output_dir="esg-binary-classifier",           # 저장될 디렉토리와 저장소 ID
    num_train_epochs=1,                      # 학습할 총 에포크 수 
    per_device_train_batch_size=4,           # GPU당 배치 크기
    gradient_accumulation_steps=2,           # 그래디언트 누적 스텝 수
    gradient_checkpointing=True,             # 메모리 절약을 위한 체크포인팅
    optim="adamw_torch_fused",               # 최적화기
    logging_steps=10,                        # 로그 기록 주기
    save_strategy="steps",                   # 저장 전략
    save_steps=50,                           # 저장 주기
    bf16=True,                              # bfloat16 사용
    learning_rate=1e-4,                     # 학습률
    max_grad_norm=0.3,                      # 그래디언트 클리핑
    warmup_ratio=0.03,                      # 워밍업 비율
    lr_scheduler_type="constant",           # 고정 학습률
    push_to_hub=False,                      # 허브 업로드 안 함
    remove_unused_columns=False,
    dataset_kwargs={"skip_prepare_dataset": True},
    report_to=None
)

`utput_dir`: 학습 결과가 저장될 디렉토리 또는 모델 저장소의 이름을 지정합니다. 이 디렉토리에 학습된 모델 가중치, 설정 파일, 로그 파일 등이 저장됩니다.

`num_train_epochs`: 모델을 학습시키는 총 에포크(epoch) 수를 지정합니다. 에포크는 학습 데이터 전체를 한 번 순회한 주기를 의미합니다. 예를 들어, `3`으로 설정하면 데이터셋을 3번 학습합니다.

`per_device_train_batch_size`: GPU 한 대당 사용되는 배치(batch)의 크기를 설정합니다. 배치 크기는 모델이 한 번에 처리하는 데이터 샘플의 수를 의미합니다. 작은 크기는 메모리 사용량이 적지만 학습 시간이 증가할 수 있습니다.

`gradient_accumulation_steps`: 그래디언트를 누적할 스텝(step) 수를 지정합니다. 이 값이 2로 설정된 경우, 두 스텝마다 그래디언트를 업데이트합니다. 배치 크기를 가상으로 늘리는 효과가 있으며, GPU 메모리 부족 문제를 해결할 때 유용합니다.

`gradient_checkpointing`: 그래디언트 체크포인팅을 활성화하여 메모리를 절약합니다. 이 옵션은 계산 그래프를 일부 저장하지 않고 다시 계산하여 메모리를 절약하지만, 속도가 약간 느려질 수 있습니다.

`optim`: 학습 시 사용할 최적화 알고리즘을 설정합니다. `adamw_torch_fused`는 PyTorch의 효율적인 AdamW 최적화기를 사용합니다.

`logging_steps`: 로그를 기록하는 주기를 스텝 단위로 지정합니다. 예를 들어, `10`으로 설정하면 매 10 스텝마다 로그를 기록합니다.

`save_strategy`: 모델을 저장하는 전략을 설정합니다. `"steps"`로 설정된 경우, 지정된 스텝마다 모델이 저장됩니다.

`save_steps`: 모델을 저장하는 주기를 스텝 단위로 설정합니다. 예를 들어, `50`으로 설정하면 매 50 스텝마다 모델을 저장합니다.

`bf16`: bfloat16 정밀도를 사용하도록 설정합니다. bfloat16은 FP32와 유사한 범위를 제공하면서 메모리와 계산 효율성을 높입니다.

`learning_rate`: 학습률을 지정합니다. 학습률은 모델의 가중치가 한 번의 업데이트에서 얼마나 크게 변할지를 결정합니다. 일반적으로 작은 값을 사용하여 안정적인 학습을 유도합니다.

`max_grad_norm`: 그래디언트 클리핑의 임계값을 설정합니다. 이 값보다 큰 그래디언트가 발생하면, 임계값으로 조정하여 폭발적 그래디언트를 방지합니다.

`warmup_ratio`: 학습 초기 단계에서 학습률을 선형으로 증가시키는 워밍업 비율을 지정합니다. 학습의 안정성을 높이기 위해 사용됩니다.

`lr_scheduler_type`: 학습률 스케줄러의 유형을 설정합니다. `"constant"`는 학습률을 일정하게 유지합니다.

`push_to_hub`: 학습된 모델을 허브에 업로드할지 여부를 설정합니다. `False`로 설정하면 업로드하지 않습니다.

`remove_unused_columns`: 사용되지 않는 열을 제거할지 여부를 설정합니다. True로 설정하면 메모리를 절약할 수 있습니다.

`dataset_kwargs`: 데이터셋 로딩 시 추가적인 설정을 전달합니다. 예제에서는 `skip_prepare_dataset: True`로 설정하여 데이터셋 준비 단계를 건너뜁니다.

`report_to`: 학습 로그를 보고할 대상을 지정합니다. `None`으로 설정되면 로그가 기록되지 않습니다.

## 4. 학습 중 전처리 함수: collate_fn

In [16]:
def collate_fn(batch):
    new_batch = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }

    for example in batch:
        messages = example["messages"]

        # LLaMA 3 채팅 템플릿 적용 (시작 토큰 포함)
        prompt = "<|begin_of_text|>"
        for msg in messages:
            role = msg["role"]
            content = msg["content"].strip()
            prompt += f"<|start_header_id|>{role}<|end_header_id|>\n{content}<|eot_id|>"

        # 마지막 assistant 메시지는 응답으로 간주하고 레이블에 포함
        text = prompt.strip()

        # 토큰화
        tokenized = tokenizer(
            text,
            truncation=True,
            max_length=max_seq_length,
            padding=False,
            return_tensors=None,
        )

        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]
        labels = [-100] * len(input_ids)

        # assistant 응답의 시작 위치 찾기
        assistant_header = "<|start_header_id|>assistant<|end_header_id|>\n"
        assistant_tokens = tokenizer.encode(assistant_header, add_special_tokens=False)
        eot_token = "<|eot_id|>"
        eot_tokens = tokenizer.encode(eot_token, add_special_tokens=False)

        # 레이블 범위 지정
        i = 0
        while i <= len(input_ids) - len(assistant_tokens):
            if input_ids[i:i + len(assistant_tokens)] == assistant_tokens:
                start = i + len(assistant_tokens)
                end = start
                while end <= len(input_ids) - len(eot_tokens):
                    if input_ids[end:end + len(eot_tokens)] == eot_tokens:
                        break
                    end += 1
                for j in range(start, end):
                    labels[j] = input_ids[j]
                for j in range(end, end + len(eot_tokens)):
                    labels[j] = input_ids[j]  # <|eot_id|> 토큰도 포함
                break
            i += 1

        new_batch["input_ids"].append(input_ids)
        new_batch["attention_mask"].append(attention_mask)
        new_batch["labels"].append(labels)

    # 패딩 처리
    max_length = max(len(ids) for ids in new_batch["input_ids"])
    for i in range(len(new_batch["input_ids"])):
        pad_len = max_length - len(new_batch["input_ids"][i])
        new_batch["input_ids"][i].extend([tokenizer.pad_token_id] * pad_len)
        new_batch["attention_mask"][i].extend([0] * pad_len)
        new_batch["labels"][i].extend([-100] * pad_len)

    for k in new_batch:
        new_batch[k] = torch.tensor(new_batch[k])

    return new_batch

입력으로 사용되는 라마 챗 템플릿은 아래와 같습니다.되어 반환됩니다.

```python
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

시스템 프롬프트<|eot_id|><|start_header_id|>user<|end_header_id|>

유저 프롬프트<|eot_id|><|start_header_id|>assistant<|end_header|>LLM의 답변<|eot_id|>
```

collate_fn(batch) 함수는 자연어 처리 모델 학습을 위해 데이터를 전처리하는 역할을 수행합니다. 이 함수는 배치 내의 데이터를 처리하여 모델이 사용할 수 있는 입력 형식으로 변환합니다.

먼저, 각 샘플의 메시지에서 개행 문자를 제거하고 필요한 정보만 남깁니다. 정리된 메시지로 텍스트를 구성하고 이를 토큰화하여 input_ids와 label_ids를 생성합니다. 레이블 데이터의 경우 실제 assistant 응답 부분을 제외한 나머지 위치는 -100으로 설정하여 손실 계산에서 제외되도록 합니다.

In [17]:
# 데이터의 최대 길이 제한
max_seq_length=8192

# collate_fn 테스트 (배치 크기 1. 즉, 데이터 1개에 대해서 전처리를 진행해본다.)
example = train_dataset[0]
batch = collate_fn([example])

print("\n처리된 배치 데이터:")
print("입력 ID 형태:", batch["input_ids"].shape)
print("어텐션 마스크 형태:", batch["attention_mask"].shape)
print("레이블 형태:", batch["labels"].shape)


처리된 배치 데이터:
입력 ID 형태: torch.Size([1, 1159])
어텐션 마스크 형태: torch.Size([1, 1159])
레이블 형태: torch.Size([1, 1159])


In [18]:
print('입력에 대한 정수 인코딩 결과:')
print(batch["input_ids"][0].tolist())

입력에 대한 정수 인코딩 결과:
[128000, 128000, 128006, 9125, 128007, 198, 16, 13, 59134, 41953, 115296, 111068, 120155, 109849, 469, 7939, 106434, 55216, 56154, 117469, 106478, 101353, 111132, 58368, 627, 17, 13, 469, 7939, 127141, 19954, 102597, 125959, 102612, 108652, 103430, 118162, 116100, 21028, 220, 605, 60861, 108652, 103430, 121990, 119884, 97237, 106478, 101353, 111132, 58368, 627, 12, 118909, 320, 83166, 8, 551, 120878, 44690, 74769, 71023, 103185, 44690, 320, 37707, 469, 2796, 59200, 705, 102888, 77535, 91586, 105078, 22035, 41820, 320, 35725, 365, 481, 12634, 25585, 705, 65677, 55421, 111516, 106304, 33931, 320, 4888, 67667, 705, 118562, 21121, 101438, 104019, 320, 54, 5642, 9744, 705, 108280, 127812, 112785, 111426, 320, 36, 1030, 7424, 18260, 5761, 11050, 705, 55216, 102252, 124476, 62060, 110685, 320, 83146, 10604, 22515, 18413, 705, 74177, 113360, 75908, 22035, 320, 50307, 1516, 36947, 705, 48918, 101438, 115299, 33931, 126110, 320, 33, 3205, 3050, 19721, 705, 103738, 104019, 320, 

In [19]:
# 디코딩된 input_ids 출력
decoded_text = tokenizer.decode(
    batch["input_ids"][0].tolist(),
    skip_special_tokens=False,
    clean_up_tokenization_spaces=False
)

print("\ninput_ids 디코딩 결과:")
print(decoded_text)


input_ids 디코딩 결과:
<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>
1. 상장사의 뉴스를 보고 ESG 관련 기사인지 판단하시오.
2. ESG 각각에 대한 핵심 키워드는 아래의 10개 키워드를 참고해서 판단하시오.
- 환경 (Environmental) : 탄소 배출 감소 (Carbon Emission Reduction), 재생 에너지 사용 (Renewable Energy Usage), 자원 효율성 (Resource Efficiency), 폐기물 관리 (Waste Management), 친환경 제품 개발 (Eco-Friendly Product Development), 기후 변화 대응 (Climate Change Mitigation), 오염 방지 (Pollution Prevention), 생물 다양성 보호 (Biodiversity Protection), 물 관리 (Water Management)
순환 경제 (Circular Economy)
- 사회 (Social) : 노동 인권 보호 (Labor Rights Protection), 다양성 및 포용성 (Diversity and Inclusion), 직원 안전 및 건강 (Employee Health & Safety), 공정한 노동 관행 (Fair Labor Practices), 커뮤니티 참여 및 개발 (Community Engagement & Development), 제품 책임 및 안전 (Product Responsibility & Safety), 공급망 책임 (Supply Chain Responsibility), 데이터 보호 및 프라이버시 (Data Protection & Privacy), 인재 개발 및 교육 (Talent Development & Education), 고객 만족도 (Customer Satisfaction)
- 지배구조 (Governance) : 이사회 독립성 (Board Independence), 

In [20]:
print('레이블에 대한 정수 인코딩 결과:')
print(batch["labels"][0].tolist())

레이블에 대한 정수 인코딩 결과:
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -1

In [21]:
# -100이 아닌 부분만 골라 디코딩
label_ids = [token_id for token_id in batch["labels"][0].tolist() if token_id != -100]

decoded_labels = tokenizer.decode(
    label_ids,
    skip_special_tokens=False,
    clean_up_tokenization_spaces=False
)

print("\nlabels 디코딩 결과 (-100 제외):")
print(decoded_labels)


labels 디코딩 결과 (-100 제외):
근거: LG에너지솔루션이 배터리 관리 토털 솔루션(BMTS) 브랜드인 'B.around'를 출시한 것은 ESG와 관련이 있습니다. 사회 측면에서는 화재 방지와 같은 안전 진단을 포함한 배터리 관리 강화는 제품 책임 및 안전을 향상시키기 위한 노력으로 볼 수 있습니다. 이는 고객의 신뢰를 높이고 제품의 안전성을 확보하는 사회적 책임 경영과 관련이 깊습니다. 또한, 기술적 혁신을 통해 전기차 배터리의 수명을 예측하고 퇴화 상태를 진단하는 것은 자원 효율성을 향상시키는 환경적 측면도 포함하고 있습니다. 
=> True<|eot_id|>


## 5. 어텐션 마스크 확인

### 배치 크기의 의미

배치 크기란 모델이 한 번에 학습하는 데이터 샘플의 수를 의미합니다. 예를 들어 배치 크기가 3이면, 모델은 세 개의 데이터 샘플을 동시에 처리합니다. 이렇게 병렬적으로 학습하면 계산 효율성이 높아지고 학습 속도가 빨라지는 이점이 있습니다.

### 텍스트 데이터의 길이 문제

자연어 처리에서 각 샘플(문장, 대화 등)은 길이가 다양합니다. 예를 들어 배치 크기 3인 경우:

- 샘플1: "인공지능이란 무엇인가요?" → [101, 4089, 8024, 6356, 102] (5 토큰)
- 샘플2: "오늘 날씨가 정말 좋네요." → [101, 3157, 2533, 4120, 2642, 8730, 6824, 102] (8 토큰)
- 샘플3: "딥러닝 모델을 학습시키는 방법을 알려주세요." → [101, 2982, 3478, 4567, 2053, 8276, 5036, 2355, 4602, 7312, 102] (11 토큰)

여기서 101과 102는 특수 토큰으로, 각각 문장의 시작과 끝을 표시합니다.

### 패딩의 필요성

신경망의 내부 연산은 고정된 크기의 입력을 요구합니다. 이 문제를 해결하기 위해 '패딩'을 사용하여 모든 샘플의 길이를 배치 내 가장 긴 샘플에 맞춥니다.

위 예시에서는 가장 긴 샘플3(11 토큰)에 맞춰 다른 샘플들에 패딩(0)을 추가합니다:

- 샘플1: [101, 4089, 8024, 6356, 102, 0, 0, 0, 0, 0, 0] (5 실제 + 6 패딩)
- 샘플2: [101, 3157, 2533, 4120, 2642, 8730, 6824, 102, 0, 0, 0] (8 실제 + 3 패딩)
- 샘플3: [101, 2982, 3478, 4567, 2053, 8276, 5036, 2355, 4602, 7312, 102] (11 실제 토큰)

이렇게 하면 모든 샘플이 동일한 길이(11)를 가지게 되어 하나의 배치로 처리할 수 있습니다.

### 어텐션 마스크의 필요성

패딩을 추가하면 모델이 어떤 토큰이 실제 내용이고 어떤 토큰이 의미 없는 패딩인지 구분해야 합니다. 이를 위해 '어텐션 마스크'를 사용합니다:

- 샘플1 마스크: [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
- 샘플2 마스크: [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]
- 샘플3 마스크: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

트랜스포머 모델의 어텐션 메커니즘은 이 마스크를 사용하여 패딩 토큰을 무시하고 실제 의미 있는 토큰에만 집중합니다. 이렇게 하면 패딩된 부분이 모델의 예측이나 학습에 영향을 미치지 않게 됩니다.

In [22]:
# 데이터의 최대 길이 제한
max_seq_length = 8192
# 0번과 1번 데이터의 길이 확인
example0 = train_dataset[0]
example1 = train_dataset[1]
# 개별 길이 확인 (토큰화 후)
tokenized0 = tokenizer(
    # 전체 처리 과정과 동일하게 전체 대화를 토큰화
    "<|begin_of_text|>" + "".join([f"<|start_header_id|>{msg['role']}<|end_header_id|>\n{msg['content'].strip()}<|eot_id|>" for msg in example0["messages"]]),
    truncation=True,
    max_length=max_seq_length,
    padding=False,
    return_tensors=None,
)
tokenized1 = tokenizer(
    # 전체 처리 과정과 동일하게 전체 대화를 토큰화
    "<|begin_of_text|>" + "".join([f"<|start_header_id|>{msg['role']}<|end_header_id|>\n{msg['content'].strip()}<|eot_id|>" for msg in example1["messages"]]),
    truncation=True,
    max_length=max_seq_length,
    padding=False,
    return_tensors=None,
)
print(f"0번 데이터 길이: {len(tokenized0['input_ids'])}")
print(f"1번 데이터 길이: {len(tokenized1['input_ids'])}")
# 배치로 처리하여 어텐션 마스크 비교
batch = collate_fn([example0, example1])
print("\n배치 처리 후:")
print(f"입력 ID 형태: {batch['input_ids'].shape}")
print(f"어텐션 마스크 형태: {batch['attention_mask'].shape}")
# 각 샘플의 어텐션 마스크 합계 (실제 토큰 수 확인)
print(f"0번 샘플 어텐션 마스크 합계: {batch['attention_mask'][0].sum().item()}")
print(f"1번 샘플 어텐션 마스크 합계: {batch['attention_mask'][1].sum().item()}")
# 0번 샘플과 1번 샘플의 어텐션 마스크가 다른지 확인
masks_different = not torch.equal(batch['attention_mask'][0], batch['attention_mask'][1])
print(f"\n0번과 1번 샘플의 어텐션 마스크가 다른가요? {masks_different}")
# 어텐션 마스크 패턴 시각화 (처음 20개와 마지막 20개 토큰)
print("\n0번 샘플 어텐션 마스크:", batch['attention_mask'][0].tolist())
print("1번 샘플 어텐션 마스크:", batch['attention_mask'][1].tolist())
# 배치 내에서 가장 긴 시퀀스 길이 구하기
max_length_in_batch = max(len(tokenized0['input_ids']), len(tokenized1['input_ids']))
print(f"\n배치 내 최대 길이: {max_length_in_batch}")
print(f"0번 샘플 어텐션 마스크 합계 (실제 토큰 수): {batch['attention_mask'][0].sum().item()}")
print(f"1번 샘플 어텐션 마스크 합계 (실제 토큰 수): {batch['attention_mask'][1].sum().item()}")
print(f"0번 샘플 어텐션 마스크 1의 개수: {batch['attention_mask'][0].sum().item()}")
print(f"0번 샘플 어텐션 마스크 0의 개수: {(batch['attention_mask'][0] == 0).sum().item()}")
print(f"1번 샘플 어텐션 마스크 1의 개수: {batch['attention_mask'][1].sum().item()}")
print(f"1번 샘플 어텐션 마스크 0의 개수: {(batch['attention_mask'][1] == 0).sum().item()}")
# 결과 검증: 긴 샘플은 모든 어텐션 마스크가 1이고, 짧은 샘플은 일부만 1이어야 함

0번 데이터 길이: 1159
1번 데이터 길이: 1782

배치 처리 후:
입력 ID 형태: torch.Size([2, 1782])
어텐션 마스크 형태: torch.Size([2, 1782])
0번 샘플 어텐션 마스크 합계: 1159
1번 샘플 어텐션 마스크 합계: 1782

0번과 1번 샘플의 어텐션 마스크가 다른가요? True

0번 샘플 어텐션 마스크: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

## 6. 전처리 이해하기

**input_ids와 labels는 어떻게 생성되는가?**

LLM 학습에서 `input_ids`와 `labels`는 모델의 학습 목표에 따라 생성됩니다. 시스템 프롬프트까지 포함하여 설명하겠습니다.

예를 들어, 다음과 같은 대화 데이터를 모델이 학습해야 한다고 가정합니다:
- 시스템 프롬프트: `당신은 친절하고 도움이 되는 AI 어시스턴트입니다.`
- 사용자 메시지: `안녕하세요, 오늘 날씨는 어떤가요?`
- 어시스턴트 응답: `안녕하세요! 오늘 날씨는 맑고 화창합니다.`

LLaMA 3에서는 다음과 같은 템플릿 구조를 사용합니다(줄바꿈 포함):

```python
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
당신은 친절하고 도움이 되는 AI 어시스턴트입니다.<|eot_id|><|start_header_id|>user<|end_header_id|>
안녕하세요, 오늘 날씨는 어떤가요?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
안녕하세요! 오늘 날씨는 맑고 화창합니다.<|eot_id|>
```

이 전체 텍스트는 토크나이저에 의해 정수 시퀀스로 변환해봅시다.  
(실제와 다르고 가정하여 정수를 맵핑하겠습니다.)

먼저 모든 특수 토큰들은 아래의 고유 ID를 가진다고 가정해봅시다.  
- <|begin_of_text|> = 토큰 ID 1
- <|start_header_id|> = 토큰 ID 2
- <|end_header_id|> = 토큰 ID 4
- 줄바꿈 = 토큰 ID 5
- <|eot_id|> = 토큰 ID 10

역할 토큰들은 아래의 고유 ID를 가진다고 가정해봅시다.  
- system = 토큰 ID 3
- user = 토큰 ID 11
- assistant = 토큰 ID 18

전체 통합된 input_ids는 다음과 같습니다:
`input_ids = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 2, 11, 4, 5, 12, 13, 14, 15, 16, 17, 10, 2, 18, 4, 5, 19, 20, 21, 22, 23, 10]`

각 부분을 분리하면:
- 시스템 프롬프트 부분: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
- 사용자 메시지 부분: [2, 11, 4, 5, 12, 13, 14, 15, 16, 17, 10]
- 어시스턴트 응답 부분: [2, 18, 4, 5, 19, 20, 21, 22, 23, 10]

모델이 예측해야 할 영역은 assistant의 응답 부분인 `안녕하세요! 오늘 날씨는 맑고 화창합니다.`에 해당하는 토큰들입니다. 따라서 `labels`는 다음과 같이 설정됩니다:

`labels = [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 19, 20, 21, 22, 23, 10]`

여기서 주목할 점:
1. 시스템 프롬프트와 사용자 메시지에 해당하는 모든 토큰(줄바꿈 포함)은 `-100`으로 마스킹됩니다.
2. 어시스턴트 헤더와 첫 줄바꿈 토큰도 `-100`으로 마스킹됩니다.
3. 실제 어시스턴트 응답 내용(19-23)과 마지막 종료 태그(10)만 원래 토큰 ID를 유지합니다.

이처럼 `labels`는 모델이 실제로 생성해야 할 출력 부분만을 포함하고, 나머지 부분은 `-100`으로 채워져 손실 계산에서 제외됩니다. 이를 통해 모델은 입력(시스템 프롬프트+사용자 질문)을 기반으로 적절한 응답을 생성하는 방법을 학습합니다.

학습 과정에서는:
1. 모델에 `input_ids` 전체를 입력으로 제공합니다.
2. 모델은 각 위치에서 다음 토큰을 예측합니다.
3. 손실 계산 시 `labels`가 `-100`이 아닌 위치에서만 오차를 계산합니다.
4. 이를 통해 모델은 주어진 맥락(시스템 프롬프트와 사용자 질문)에 대해 적절한 응답을 생성하는 방법을 학습합니다.

## 7. 학습하기

In [23]:
trainer = SFTTrainer(
    model=model,
    args=args,
    max_seq_length=max_seq_length,  # 최대 시퀀스 길이 설정
    train_dataset=train_dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


- **학습 데이터 크기**: 3728개

모델 학습에 사용되는 전체 데이터 샘플의 수입니다. 이는 각 에포크마다 처리되는 전체 데이터 양을 의미합니다.

- **에포크(epochs)**: 1회

전체 데이터셋을 처음부터 끝까지 반복해서 학습하는 횟수입니다. 즉, 모든 학습 데이터를 1번 반복해서 모델이 학습한다는 의미입니다.

- **배치 크기(batch size)**: 4

한 번에 처리하는 데이터 샘플의 수입니다. 메모리 효율을 위해 전체 데이터를 작은 배치로 나누어 처리하며, 여기서는 4개씩 묶어서 처리합니다.

- **누적 단계(accumulation steps)**: 2

모델을 실제로 업데이트하기 전에 여러 배치의 정보를 모으는 수입니다. 여기서는 2개의 배치(총 8개의 샘플)를 처리한 후에야 실제 모델 업데이트가 일어납니다.

- **에포크 1회당 업데이트 횟수**: 3728 ÷ (4 × 2) = 466회

한 에포크에서 모델이 업데이트되는 횟수입니다. 전체 데이터 3728개를 유효 배치 크기 8(배치 크기 4 × 누적 단계 2)로 나누면 466번의 업데이트가 발생합니다.

- **총 업데이트 계산 방법**: (데이터 크기 × 에포크) ÷ (배치 크기 × 누적 단계)

학습 과정 전체에서 발생하는 모델 업데이트의 총 횟수를 계산하는 공식입니다. 전체 처리 샘플 수를 유효 배치 크기로 나눕니다.

- **총 업데이트 계산 과정**: (3728 × 1) ÷ (4 × 2) = 3728 ÷ 8 = 466

1개의 에포크 동안 총 3728개의 샘플이 처리되고, 유효 배치 크기인 8개의 샘플마다 한 번씩 모델이 업데이트되므로 총 466의 모델 업데이트가 발생합니다.

In [ ]:
# 학습 시작
trainer.train()   # 모델이 자동으로 허브와 output_dir에 저장됨

# 모델 저장
trainer.save_model()   # 최종 모델을 저장

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
10,1.149300
20,0.934900
30,0.876100
40,0.842100
50,0.827900


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


## 8. 테스트 데이터 준비하기

실제 모델에 입력을 넣을 때에는 입력의 뒤에 `<|start_header_id|>assistant<|end_header_id|>\n`가 부착되어서 넣는 것이 좋습니다. 그러면 모델이 조금 더 안정적으로 답변을 생성합니다.

In [ ]:
prompt_lst = []
label_lst = []

for messages in test_dataset["messages"]:
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    input = text.split('<|start_header_id|>assistant<|end_header_id|>\n')[0] + '<|start_header_id|>assistant<|end_header_id|>\n'
    label = text.split('<|start_header_id|>assistant<|end_header_id|>\n')[1].split('<|eot_id|>')[0]
    prompt_lst.append(input)
    label_lst.append(label)

In [ ]:
print(prompt_lst[200])

In [ ]:
print(label_lst[200])

## 9. 파인튜닝 모델 테스트

`AutoPeftModelForCausalLM()`의 입력으로 LoRA Adapter가 저장된 체크포인트의 주소를 넣으면 LoRA Adapter가 기존의 LLM과 부착되어 로드됩니다. 이 과정은 LoRA Adapter의 가중치를 사전 학습된 언어 모델(LLM)에 통합하여 미세 조정된 모델을 완성하는 것을 의미합니다.

`peft_model_id` 변수는 미세 조정된 가중치가 저장된 체크포인트의 경로를 나타냅니다. `"llama3-8b-text-to-sql/checkpoint-1875"`는 LoRA Adapter 가중치가 저장된 위치로, 이 경로에서 해당 가중치를 불러옵니다.

`fine_tuned_model`은 `AutoPeftModelForCausalLM.from_pretrained` 메서드를 통해 체크포인트를 로드하여 생성됩니다. 이 메서드는 LLM과 LoRA Adapter를 결합하고, 최적화된 설정으로 모델을 메모리에 로드합니다. `device_map="auto"` 옵션은 모델을 자동으로 GPU에 배치합니다.

`pipeline`은 Hugging Face의 고수준 유틸리티로, NLP 작업(예: 텍스트 생성, 번역, 요약 등)을 간단히 수행할 수 있게 해줍니다. 이 코드에서 사용된 `pipeline("text-generation")`은 텍스트 생성 작업을 수행하기 위한 파이프라인 객체를 생성합니다. 파이프라인은 내부적으로 모델과 토크나이저를 관리하여, 입력 텍스트를 토큰화하고, 모델을 통해 생성된 결과를 다시 디코딩하여 사람이 읽을 수 있는 텍스트로 변환합니다.

이 코드는 미세 조정된 LLM을 로드한 뒤, 이를 이용해 텍스트 생성 작업을 간단히 수행할 수 있도록 준비하는 데 목적이 있습니다. `pipeline`을 통해 텍스트 생성 작업을 실행하면, 입력 텍스트에 기반하여 모델이 다음 토큰을 예측하고 이를 반복적으로 생성합니다. 이 과정은 사용자에게 자연스러운 텍스트를 출력하는 데 사용됩니다.데 사용됩니다.

In [27]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import  AutoTokenizer, pipeline

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [ ]:
peft_model_id = "esg-binary-classifier/checkpoint-466"
fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id, device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=fine_tuned_model, tokenizer=tokenizer)

In [ ]:
eos_token = tokenizer("<|eot_id|>",add_special_tokens=False)["input_ids"][0]

In [ ]:
def test_inference(pipe, prompt):
    outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=eos_token, do_sample=False)
    return outputs[0]['generated_text'][len(prompt):].strip()

In [ ]:
for prompt, label in zip(prompt_lst[10:15], label_lst[10:15]):
    # print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)

## 10. 기본 모델 테스트

이번에는 LoRA Adapter를 merge하지 않은 기본 모델로 테스트 데이터에 대해서 인퍼런스해보겠습니다.


In [ ]:
base_model_id = "allganize/Llama-3-Alpha-Ko-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(base_model_id, device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
for prompt, label in zip(prompt_lst[10:15], label_lst[10:15]):
    # print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)